In [1]:
import pandas as pd
import json
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import os
import time

In [ ]:
def to_int(val):
    """ Reconoce valores numericos y los transforma a enteros.
    """
    try:
        value = int(float(val))
    except ValueError:
        value = ""
    return value



In [16]:
df = pd.read_csv("champions_league_fixtures.csv")
df

,date,time_or_status,home_team,away_team,match_id,home_score,away_score,home_odds,draw_odds,away_odds
0,"Tuesday, Oct 01 2024",FT,Salzburg,Brest,1866220,0,4,NaN,NaN,NaN
1,"Tuesday, Oct 01 2024",FT,VfB Stuttgart,Sparta Prague,1866237,1,1,NaN,NaN,NaN
2,"Tuesday, Oct 01 2024",FT,PSV Eindhoven,Sporting CP,1866209,1,1,NaN,NaN,NaN
3,"Tuesday, Oct 01 2024",FT,Arsenal,Paris Saint-Germain,1866166,2,0,NaN,NaN,NaN
4,"Tuesday, Oct 01 2024",FT,Slovan Bratislava,Manchester City,1866104,0,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
85,"Thursday, Sep 19 2024",FT,FK Crvena Zvezda,Benfica,1866185,1,2,NaN,NaN,NaN
86,"Thursday, Sep 19 2024",FT,Brest,Sturm Graz,1866240,2,1,NaN,NaN,NaN
87,"Thursday, Sep 19 2024",NaN,Atalanta,Arsenal,1866171,0,0,NaN,NaN,NaN
88,"Thursday, Sep 19 2024",FT,Monaco,Barcelona,1866140,2,1,NaN,NaN,NaN


In [17]:
def transform_date(date_str):
    # date_obj = datetime.strptime(date_str, "%A, %b %d %Y")
    # return date_obj.strftime("%Y%m%d")
    cleaned_date = re.search(r'\b\w{3} \d{2} \d{4}\b', date_str).group()
    print(cleaned_date)
    return cleaned_date

# Crear directorio para guardar los archivos JSON
os.makedirs("partidos", exist_ok=True)

# Leer el CSV
df = pd.read_csv("champions_league_fixtures.csv")
df["match_id"] = df["match_id"].map(to_int)

# Configurar opciones de Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")  # Comentar esta línea si quieres ver el navegador
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36")

# Inicializar el servicio de Chrome
service = Service(ChromeDriverManager().install())

# Inicializar el driver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Contador de exportaciones
total_exportaciones = 0

# Iterar sobre cada fila del DataFrame
for index, row in df.iterrows():
    match_id = row['match_id']
    url = f'https://es.whoscored.com/Matches/{match_id}/Live/'
    
    print(f"Procesando partido: {row['home_team']} vs {row['away_team']} (ID: {match_id})")
    
    try:
        # Acceder a la página
        driver.get(url)
        
        # Esperar un poco para que se cargue el contenido dinámico
        time.sleep(5)
        
        # Obtener el contenido de la página
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Buscar el script que contiene los datos
        scripts = soup.find_all("script")
        datos = ''
        for script in scripts:
            if script.string and "matchCentreData" in script.string:
                datos = script.string
                break
        
        if datos:
            # Extraer los datos JSON
            listVar = datos.split(';')
            matchCenterData = listVar[0].split('require.config.params["args"] = ')[1]
            matchCenterData = matchCenterData.replace('\n','').replace('matchId','"matchId"')
            matchCenterData = matchCenterData.replace('matchCentreData','"matchCentreData"')
            matchCenterData = matchCenterData.replace('formationIdNameMappings','"formationIdNameMappings"')
            matchCenterData = matchCenterData.replace('matchCentreEventTypeJson','"matchCentreEventTypeJson"')
            
            # Cargar los datos JSON
            data = json.loads(matchCenterData)
            
            # Crear el nombre del archivo
            file_name = f"{transform_date(row['date'])}_{row['home_team']}_{row['away_team']}_{row['match_id']}.json"
            
            # Guardar el JSON
            with open(os.path.join("partidos", file_name), 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)
            
            print(f"Archivo exportado: {file_name}")
            total_exportaciones += 1
        else:
            print(f"No se encontraron datos para el partido {match_id}")
    except Exception as e:
        print(f"Error al procesar el partido {match_id}: {str(e)}")

# Cerrar el driver
driver.quit()

# Imprimir el total de exportaciones
print(f"\nTotal de partidos exportados: {total_exportaciones}")

Procesando partido: Salzburg vs Brest (ID: 1866220)
Oct 01 2024
Archivo exportado: Oct 01 2024_Salzburg_Brest_1866220.json
Procesando partido: VfB Stuttgart vs Sparta Prague (ID: 1866237)
Oct 01 2024
Archivo exportado: Oct 01 2024_VfB Stuttgart_Sparta Prague_1866237.json
Procesando partido: PSV Eindhoven vs Sporting CP (ID: 1866209)
Oct 01 2024
Archivo exportado: Oct 01 2024_PSV Eindhoven_Sporting CP_1866209.json
Procesando partido: Arsenal vs Paris Saint-Germain (ID: 1866166)
Oct 01 2024
Archivo exportado: Oct 01 2024_Arsenal_Paris Saint-Germain_1866166.json
Procesando partido: Slovan Bratislava vs Manchester City (ID: 1866104)
Oct 01 2024
Archivo exportado: Oct 01 2024_Slovan Bratislava_Manchester City_1866104.json
Procesando partido: Inter vs FK Crvena Zvezda (ID: 1866108)
Oct 01 2024
Archivo exportado: Oct 01 2024_Inter_FK Crvena Zvezda_1866108.json
Procesando partido: Borussia Dortmund vs Celtic (ID: 1866131)
Oct 01 2024
Archivo exportado: Oct 01 2024_Borussia Dortmund_Celtic_1866